# Full Walkthrough

## Import Data from Kaggle

In [ ]:
%%bash
export KAGGLE_USERNAME=username
export KAGGLE_KEY=key
kaggle datasets download -d prithwirajmitra/covid-face-mask-detection-dataset
kaggle datasets download -d andrewmvd/face-mask-detection

## Load and Preprocess Data

In [ ]:
# Unzip downloaded zip files and load them into directories

import zipfile

with zipfile.ZipFile("covid-face-mask-detection-dataset.zip", "r") as zipped:
    zipped.extractall("face-mask-data")
with zipfile.ZipFile("face-mask-detection.zip", "r") as zipped:
    zipped.extractall("face-detection-data")

In [ ]:
# Create a list of paths to our image data

from os import listdir
from os.path import isfile, join

# Instantiate list
path_list = []
base_path = "face-mask-data/New Masks Dataset"

# Data was split into 3 modes
for mode in ["Train", "Test", "Validation"]:

    # Data of each mode is split into mask or no mask
    for is_mask in ["Mask", "Non Mask"]:

        # Add all paths in this directory into our path_list
        # Each item includes the path to the image and the label if it is masked or not
    path_list.extend([[f"{base_path}/{mode}/{is_mask}/{f}", [0,1] if is_mask=="Mask" else [1,0], mode] \
        for f in listdir(f"{base_path}/{mode}/{is_mask}") if isfile(join(f"{base_path}/{mode}/{is_mask}/{f}"))])
path_list[:5]


In [ ]:
!pip3 install mtcnn

In [ ]:
# Import necessary libraries
from PIL import Image
from mtcnn import MTCNN
import numpy as np, pandas as pd
import cv2

# Define uniform image size
image_x = 299
image_y = 299


def preprocess_image(image, net_h, net_w):
    new_h, new_w, _ = image.shape

    # determine the new size of the image
    if (float(net_w)/new_w) < (float(net_h)/new_h):
        new_h = (new_h * net_w)//new_w
        new_w = net_w
    else:
        new_w = (new_w * net_h)//new_h
        new_h = net_h
    
    # Try resizing image, sometimes there are images that trigger errors related to cv2's source code, which we can filter out
    try:
      resized = cv2.resize(image/255., (int(new_w), int(new_h)))
      
      # Create gray background with right size
      new_image = np.ones((net_h, net_w, 3)) * 0.5

      # Overlay resized image on gray background, essentially padding the image to fit the net size
      new_image[int((net_h-new_h)//2):int((net_h+new_h)//2), int((net_w-new_w)//2):int((net_w+new_w)//2), :] = resized
      new_image = np.expand_dims(new_image, 0)
      return new_image
    except Exception as e:
      print(e)
      return

def data_parse(files):
  data_dict = {}

  # Instantiate face detector MTCNN,
  # which is a Multi-Task Convolusional Neural Network.
  # Learn more at: https://github.com/ipazc/mtcnn
  detector = MTCNN()

  # Create separat datasets for each mode.
  # This allows for us to train, test, and validate our model
  # on different sets of data
  for mode in ['Test', 'Train', 'Validation']:

    # Create the features and label lists for each mode
    data_dict[mode] = [[],[]]
  
  # Loop through each image
  for i,f in enumerate(files):

    # Counter to show progress
    if i % 100 == 0:
      print(i)

    # Load in image as a 3D array (d1 is y, d2 is x, d3 is RGB)
    im = cv2.cvtColor(cv2.imread(f[0]), cv2.COLOR_BGR2RGB)

    # Find all faces in image
    faces = detector.detect_faces(im)

    # Loop through each face
    for face in faces:

      # Only take the part of the image that is detected to be a face
      far_x = face['box'][0]+face['box'][2]
      far_y = face['box'][1]+face['box'][3]
      im_arr = im[face['box'][1]:far_y, face['box'][0]:far_x, :3]
      if 0 in im_arr.shape:
        continue
      # Pad image to make it a uniform size (299x299) as defined above
      process_res = preprocess_image(im_arr, image_y, image_x)
      if process_res is not None:

        # Append image and label to the appropriate mode
        data_dict[f[2]][0].append(process_res)
        data_dict[f[2]][1].append(f[1])
  return data_dict

d = {}

# Check if there is an already trained model
if not isfile('/content/mask_predictor.h5'):
  d = data_parse(path_list)

In [ ]:
final_dict = {}
if not isfile('/content/mask_predictor.h5'):


  # Format to "X/y_Test/Train/Validation"
  for mode in ['Test', 'Train', 'Validation']:
    final_dict[f"X_{mode}"] = np.array(d[mode][0])[:, 0, :, :, :]
    final_dict[f"y_{mode}"] = np.array(d[mode][1])

In [ ]:
import matplotlib.pyplot as plt

if not isfile('/content/mask_predictor.h5'):

  # First X_Train image
  plt.imshow(final_dict['X_Train'][0])
  plt.show()

## Model Definition with Transfer Learning Base Model

In [ ]:
from keras.applications import Xception
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D, Flatten
import tensorflow as tf

In [ ]:
# Use Xception Model as a base model
base_model = Xception(include_top=False, input_shape=(image_x, image_y,3))

# Freeze weights of Xception because Xception is already really good at identifying images
base_model.trainable = False

# Create Sequential model based on Xception
model = Sequential([
                    # Xception
                    base_model,

                    # Turn 2D to 1D
                    Flatten(),

                    # 2 units correspond to mask% and no_mask%
                    Dense(2, activation='sigmoid')
])


# Compile model
model.compile(
    loss='binary_crossentropy', # Binary_crossentropy because we are doing binary classification
    optimizer='adam', # Adam is a generally strong optimizer for image processing
    metrics=[tf.keras.metrics.BinaryAccuracy()] # Measure success with binary accuracy
)

model.summary()

## Model Training

In [ ]:
# Define some necessary parameters
EPOCHS = 3 # How many rounds of training we want to do
BATCH_SIZE = 32 # The size of one batch of data

if isfile('/content/mask_predictor.h5'):
  # Load model from file if file exists
  model = keras.models.load_model('/content/mask_predictor.h5')
else:
  # Train model with .fit() method
  model.fit(final_dict["X_Train"], final_dict['y_Train'], validation_data=(final_dict["X_Validation"], final_dict["y_Validation"]), epochs=EPOCHS, batch_size=BATCH_SIZE)

## Model Evaluation

In [ ]:
# Evaluate model success with .evaluate() method
loss, acc = model.evaluate(final_dict["X_Test"], final_dict["y_Test"], batch_size=BATCH_SIZE)

In [ ]:
# Show Loss and Accuracy
print(f"LOSS {loss}")
print(f"ACCURACY : {acc}")

## Create Prediction Serving Preprocessing Functions

In [ ]:
def preprocess_serving(f):

  # Instantiate image list
  X = []
  

  # Instantiate MTCNN detector
  # Learn more at: https://github.com/ipazc/mtcnn
  detector = MTCNN()

  # # Loop through each provided file path
  # for f in files:

  # Read image as 2D numpy array
  im = cv2.cvtColor(cv2.imread(f), cv2.COLOR_BGR2RGB)

  # Detect faces in image
  faces = detector.detect_faces(im)
  if len(faces) == 0:
    print(f"File {f} Contains No Faces")
    return

  # Loop through each face
  for face in faces:

    # Crop image to contain one face
    far_x = face['box'][2] + face['box'][0]
    far_y = face['box'][3] + face['box'][1]
    im_arr = im[face['box'][1]:far_y, face['box'][0]:far_x, :3]
    if 0 in im_arr.shape:
      print("Invalid Face, Please Try Another Image")
      return

    # Pad image to fit correct size
    process_res = preprocess_image(im_arr, image_y, image_x)
    if process_res is None:
      print("Invalid Face, Please Try Another Image")
      return

    X.append(process_res)
  return np.array(X), faces

In [ ]:
from PIL import Image, ImageDraw

In [ ]:
# Draw face boxes on image to identify mask or not
def draw_faces(f, boxes, preds, confidence=1):
  im = Image.open(f)
  imDraw = ImageDraw.Draw(im)
  for i, box in enumerate(boxes):
    color = "green" if preds[i][0] else "red"
    imDraw.rectangle([(box[0], box[1]), (box[0]+box[2], box[1]+box[3])], outline=color)
    if confidence:
      imDraw.rectangle([(box[0], box[1]), (box[0]+120, box[1]-20)], fill=color)
      imDraw.text([box[0]+2, box[1]-15], f"Confidence: {preds[i][1]:.2f}", fill="white")

  return im

## Establishing Pipeline

### Steps:
###   - get path to image
###   - preprocess image with serving_parse()
###   - predict all returned data with model

In [ ]:
from IPython.display import display
import time

In [ ]:
# Time the entire operation
start_time = time.time()

# Get path to image
given_path = "/content/test_image3.jpg"

# Preprocess image with data_parse() and get all faces in image
data, faces = preprocess_serving(given_path)

# Reshape data for prediction convenience
data= data.reshape(data.shape[0], image_y, image_x, 3)

# Predict data with model
pred_prob = model.predict(data)

# Do basic response and create binary prediction list
preds = []
for prob in pred_prob:
  pred = np.argmax(prob)
  preds.append([pred, prob[pred]])
  if pred:
    print("You are wearing a mask")
  else:
    print("You are not wearing a mask")
print(f"Time of Image Detection and Prediction: {time.time()-start_time:.2f}")
second_time = time.time()

# Draw bounding boxes of faces on image
im_check = draw_faces(given_path, [i['box'] for i in faces], preds, confidence=1)


display(im_check)

third_time = time.time()
print(f"Time of Image Drawing: {third_time-second_time:.2f}")